Credit: https://exnrt.com

How to Fine-tune Llama-3 8B: https://exnrt.com/blog/ai/finetune-llama3-8b/

How to Fine-tune Mistral 7B: https://exnrt.com/blog/ai/mistral-7b-fine-tuning/

How to Fine-tune Gemma: https://exnrt.com/blog/ai/finetune-gemma-with-huggingface-transformers/
<br><br>
<div>
  <a href="https://huggingface.co/Ateeqq"><img src="https://huggingface.co/front/assets/huggingface_logo-noborder.svg" width="70"></a>
</div>
Follow Me

In [ ]:
!pip install -U datasets accelerate peft wandb

In [ ]:
# Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
from transformers import Trainer, TrainingArguments

In [ ]:
# Define model and tokenizer
model_id = "meta-llama/Meta-Llama-3-8B"
#model_id = "meta-llama/Meta-Llama-3-8B-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [ ]:
# Load dataset
dataset = load_dataset("ruslanmv/ai-medical-chatbot")

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

In [ ]:
def preprocess_function(examples):
    instructions = examples['Patient']
    responses = examples['Doctor']
    return tokenizer.prepare_seq2seq_batch(
        src_texts=instructions,
        tgt_texts=responses,
        return_tensors='pt',
        max_length=1024,  # Adjust the max length as needed
        truncation=True  # Enable truncation
    )

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/256916 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4047: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is depr

In [ ]:
for_instruct_test = """def preprocess_function(examples):
    instructions = [{"role": "system", "content": "You are a helpful medical chatbot."}] + \
                   [{"role": "user", "content": description} for description in examples['Description']]
    responses = [{"role": "assistant", "content": response} for response in examples['Doctor']]
    return tokenizer.prepare_seq2seq_batch(
        src_texts=[str(instruction) for instruction in instructions],
        tgt_texts=[str(response) for response in responses],
        max_length=1024,  # Adjust the max length as needed
        truncation=True,
        #return_overflowing_tokens=True # Enable truncation
    )"""

#tokenized_dataset = dataset.map(preprocess_function, batched=True, batch_size=9)

In [ ]:
for_instruct_test2 = """def preprocess_function(examples):
    instructions = [{"role": "system", "content": "You are a helpful medical chatbot."}] + \
                   [{"role": "user", "content": description} for description in examples['Description']]
    responses = [{"role": "assistant", "content": response} for response in examples['Doctor']]

    model_inputs = tokenizer(
        [str(instruction) for instruction in instructions],
        padding="max_length",  # Example padding setting
        truncation=True,       # Example truncation setting
        max_length=512         # Example max_length setting
    )

    labels = tokenizer(
        text_target=[str(response) for response in responses],
        padding="max_length",  # Example padding setting
        truncation=True,       # Example truncation setting
        max_length=512         # Example max_length setting
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs"""
  #tokenized_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./finetuned_model",
    num_train_epochs=3,           # Adjust based on your needs
    per_device_train_batch_size=1,  # Adjust based on your hardware
    save_steps=10000,             # Adjust based on your preferences
    eval_steps=5000,              # Adjust based on your preferences
    # Add additional arguments like learning rate, weight decay, etc.
)

In [ ]:
# Check if any modules are offloaded to CPU or disk
!nvidia-smi

Mon Apr 22 23:09:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   49C    P0              28W /  72W |  15687MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    # eval_dataset=tokenized_dataset["validation"],
    # Add data collator if needed
)

In [ ]:
import torch
import os
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [ ]:
# Start training
trainer.train()

# Save the fine-tuned model
trainer.save_model("./finetuned_model")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# Example of using the fine-tuned model
messages = [
    {"role": "system", "content": "You are a helpful medical chatbot."},
    {"role": "user", "content": "I have a headache and fever."},
]

input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
outputs = model.generate(input_ids, max_new_tokens=256, do_sample=True, temperature=0.7)
response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
print(response)